네이버 영화 TOP 100 데이터 크롤링하기

라이브러리 설치

In [ ]:
!pip install selenium

라이브러리 import

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

분석할 데이터 모으기(크롤링)

In [ ]:
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=(1,5) 실시간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page=(1,5) 일간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page=(1,5) 주간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=(1,5) 월간

def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] # 100개의 영화 링크를 수집할 리스트 변수 선언

  for pageNo in range(startPageNo - 1, lastPageNo) :
    # 실시간 TOP 100 영화 링크 수집
    url = 'https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=' + str(pageNo + 1)

    req = requests.get(url) # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml') # html 파싱을 위한 객체 생성
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">

    for movielink in movielinks :
      link = str(movielink.get('href'))
      links.append('https://series.naver.com'+link) # 접속할 수 있는 전체 링크 형태로 변환해서 append

  return links

네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links) :
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = "http://www.naver.com"

  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url2)
  time.sleep(3.0) # 3초 sleep

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't')

  for link in links :
    print(link + ' 수집 중......')
    driver.switch_to.window(driver.window_handles[-1]) # 새 탭 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(3.0)

    html_source = driver.page_source # req.text 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년관람불가 영화 크롤링할 경우 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = ''.join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버' :
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em ')
      score = float(score.text)
      score = int(score)
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르','')
      genre = genre.replace('\n','')
      genre = genre.replace('\t','')
      genre_infos.append(genre)

      # 제목, 줄거리
      try :
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except :
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')
      
      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text,'lxml')
      title = movie_soup.head.find('meta',{'property':'og:title'}).get('content')
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0 :
        content_infos.append('줄거리 오류')

      else :
        for contents in contents_texts :
          # 줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace('\r','') # 줄 바꿈 제거
          temp = temp.replace('\xa0','') # 공백 제거
          content_infos.append(temp)
    elif newflag == '' :
      print('청불 영화로 데이터 수집하지 않습니다.')
  print('수집 완료합니다......')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {'제목':title_infos, '평점':score_infos, '장르':genre_infos, '줄거리':content_infos}
    
  # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates('줄거리', keep='first') # 줄거리 비교해서, 중복된 영화를 삭제(첫번째 수집 영화만 남김)

  return movie_df2

csv 파일로 저장

In [ ]:
def dftocsv(movie_df, num) :
  try :
    movie_df.to_csv(('month_movie_my_data_'+ str(num) +'.csv'),
                    sep=',', na_rep='NaN', encoding='euc-kr')
  except :
    print('Error')

크롤링 실행

In [ ]:
# 1) 크롤링 할 링크 수집
links = getPageLinksWantRange(1,5)

# 2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

# 3) csv 파일 저장
dftocsv(movie_df2, len(movie_df2))

https://series.naver.com/movie/detail.nhn?productNo=6119777 수집 중......
7
https://series.naver.com/movie/detail.nhn?productNo=6176165 수집 중......
8
https://series.naver.com/movie/detail.nhn?productNo=6158879 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6098871 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6023769 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6139377 수집 중......
8
https://series.naver.com/movie/detail.nhn?productNo=6139375 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6119778 수집 중......
7
https://series.naver.com/movie/detail.nhn?productNo=4356726 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6176518 수집 중......
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3293948 수집 중......
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6039515 수집 중......
0
https://series.naver.com/movie/detail.nhn?productNo=6176542 수집 중......
8
https://ser